<a href="https://colab.research.google.com/github/Cyusa8/ML-projects/blob/main/project2(Project_priority_level_prediction).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install keras~=3.8.0 \
matplotlib~=3.10.0 \
  numpy~=2.0.0 \
  pandas~=2.2.0 \
  tensorflow~=2.18.0
print("Install was successful")

Install was successful


In [18]:
  #load required modules
#general module
import sys
import io
import zipfile
import requests
from io import BytesIO
#import data module
import numpy as np
import pandas as pd
#import machine learning module
import keras
#data visualisation module
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
training_df = pd.read_csv("/content/drive/MyDrive/large_project_priority_data.csv")
raw_df = training_df[['Estimated Cost (in $)','Priority Value','Community Benefit Score','Stakeholder Engagement Score']]
a = len(raw_df .index)
#now we are going to set nbr of rows of our table (dataframe => training_df) calculates and dispay nbr of rows and .format inserts the value in the string
print(f'Table number of rows:{a}\n\n')
#this code displays 200 rows of our table starting from the head
raw_df.describe(include='all')

Table number of rows:1000




,Estimated Cost (in $),Priority Value,Community Benefit Score,Stakeholder Engagement Score
count,1000.000000,1000.000000,1000.000000,1000.00000
mean,245679.568000,49.312828,50.089000,48.44600
std,140316.549896,16.356126,29.309557,29.21761
min,10051.000000,1.949479,0.000000,0.00000
25%,122890.250000,38.457918,24.000000,24.00000
50%,242653.000000,49.711113,51.000000,48.00000
75%,368214.250000,61.126907,75.000000,73.00000
max,499910.000000,95.495823,100.000000,100.00000


In [16]:
raw_df.corr(numeric_only = True)

,Estimated Cost (in $),Priority Value,Community Benefit Score,Stakeholder Engagement Score
Estimated Cost (in $),1.000000,0.488671,-0.002967,-0.044831
Priority Value,0.488671,1.000000,0.687992,0.476354
Community Benefit Score,-0.002967,0.687992,1.000000,-0.050878
Stakeholder Engagement Score,-0.044831,0.476354,-0.050878,1.000000


In [12]:

raw_df ['Priority_value(%)'] =  (raw_df ['Priority Value']*100/100).map(lambda x: f"{x:.2f}%")
raw_df ['Community_Benefit(%)'] = (raw_df ['Community Benefit Score']*100/100).map(lambda x: f"{x:.2f}%")
raw_df ['Stakeholder_Engagement(%)'] =  (raw_df ['Stakeholder Engagement Score']*100/100).map(lambda x: f"{x:.2f}%")
df = raw_df [['Community_Benefit(%)','Stakeholder_Engagement(%)', 'Estimated Cost (in $)','Priority_value(%)']]
df.head(200)


,Community_Benefit(%),Stakeholder_Engagement(%),Estimated Cost (in $),Priority_value(%)
0,9.00%,97.00%,220740,45.95%
1,87.00%,31.00%,290891,61.56%
2,33.00%,10.00%,496932,46.02%
3,3.00%,6.00%,325813,22.55%
4,69.00%,25.00%,214193,47.95%
...,...,...,...,...
195,1.00%,13.00%,156607,13.70%
196,54.00%,61.00%,469761,68.09%
197,23.00%,6.00%,83026,15.98%
198,24.00%,94.00%,154403,47.07%


We define plots

We will be defining functions for; Building training and fitting of our model through experiments.

In [30]:
def build_model(num_features,my_learning_rate):
  ''' Am going to define functions of building my linear regression models which is  built on a neural network.
      A neural network is a model that works in a way that mimics the way a human brain neuron works
      A neural network contains of layers of nodes and each node is treated as its own model which consists  of
      an input layer a hidden layer which maybe more than one, and an output layer, with each node having its own bias and threshold.
       It helps when training a model where each node's output is compared to the threshold and bias before it is passed  on
       to the next  node and that goes on until the size of the sample is exhausted. So we create a neural network for our linear
       regression model  with build_model()function'''
  #we define input and output layers
  inputs = keras.Input(shape=(num_features,))
  outputs = keras.layers.Dense(units=1)(inputs)
  model   = keras.Model(inputs=inputs, outputs=outputs)
  ''' optimizer is used as a way how gradient descent adjust parameters and parameters needs to be adjusted during training for a highly probable prediction '''
  model.compile(optimizer = keras.optimizer.RMSprop(learning_rate = my_learning_rate),
                loss = 'mean_squred_error',
                metrics=[keras.metrics.RootMeanSquaredError()])
  return model

def train_model(model, df, features, label, epochs, batchsize):
  ''' model.fit is like the heart of prediction since it adjusts the model to perfect predictions
      this model.fit method trains the model and the results are stored in history variable'''
  history = model.fit(x=features, y=label, epochs=epochs, batch_size=batchsize  )
  #We extract our parameters from our model we built above by calling keras api for NNs architected models
  trained_weigths = model.get_weights()[0]
  trained_bias = model.get_weights()[1]
  #history.epochs: Retrieves the number of epochs the model was trained for and stored in epochs.
  epochs = history.epochs
  #pd.DataFrame() this takes in dictionary 'a copy of my history' and transforms into a more organised DataFrame
  hist_df = pd.DataFrame(history.history)
  #A generating rmse from my new dataframe hist_df
  rmse = hist_df['root_mean_squared error']

  return trained_bias, trained_weigths, rmse, epochs, batchsize

def experiment(df, feature_names, label_name, epochs, batch_size, learning_rate):
  a = feature_names
  b = label_name
  print(f'Info: Start training model with feature={a} and label={b}\n')

  ''' am extracting feature_names from my dataframe(df) and with .value I convert it into a numpy array to help in math computations
      df.loc locates and extracts all columns with feature_names as a name of column and the ':' extract all rows
      and features stores it same method as used down for extracting the label to base on predictions but we didn't use .loc since
      only have a single target column  '''
  features = df.loc[:,feature_names].value
  label    = df[label_name].value

  model = build_model(features,learning_rate)
  model_output = train_model(model, df, features, label, epochs, batch_size)

#Display results and visualisation
  print('\n SUCCESS:Training Experiment Complete\n')
  #Return trained model
  return model

print("SUCCESS: defining linear regression functions complete.")



SUCCESS: defining linear regression functions complete.
